In [1]:
 pip install scipy scikit-image libigl pygeodesic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import igl
import scipy
import matplotlib.pyplot as plt
import pygeodesic.geodesic as geodesic


def visualize_mat(mat, title, output_path):
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    plt.matshow(mat)
    ax.axis('equal')
    plt.colorbar()
    plt.title(title)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path)


def load_landmark_file(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    vids = np.array([int(line.strip().split(',')[0]) for line in lines])
    names = [line.strip().split(',')[1] for line in lines]

    return vids, names

def energy_sample_generator(method, emin, emax, nsamples, variance=5):
    if method == 'log_linear':
        E = np.linspace(np.log(emin), np.log(emax) / 1.02, nsamples)
        sigma = (E[1] - E[0]) * variance
        return E, sigma
    elif method == 'log_sampled':
        tmin = np.abs(4 * np.log(10) / emax)
        tmax = np.abs(4 * np.log(10) / emin)
        E = np.exp(np.linspace(np.log(tmin), np.log(tmax), nsamples))
        return E
    else:
        assert 0, f'Unsupported sample method {method}'


def wave_kernel_signature(evecs, evals, energies, sigma):
    # Computes the wave kernel signature according to the spectrum of a graph
    # derived operator(e.g., the Cotangent Laplacian).
    # This signature was introduced in the paper of M.Aubry, U.Schlickewei and D.Cremers
    # "The Wave Kernel Signature: A Quantum Mechanical Approach To Shape Analysis"
    # http: // www.di.ens.fr / ~aubry / texts / 2011 - wave - kernel - signature.pdf
    #
    # Input: evecs - [n, k] Eigenvectors of a graph operator
    # n is the number of nodes of the graph.
    # evals - [k] Corresponding eigenvalues.
    # energies - [e] Energy values over which the kernel is
    # evaluated.
    # sigma - float Controls the variance of the fitted gausian.
    #
    # Output: signatures - [n, e] Matrix with the values of the WKS for
    # different energies in its columns.

    gaussian_kernel = np.exp(-((energies.reshape(1, -1) - np.log(evals).reshape(-1, 1)) ** 2) / (2 * sigma ** 2))
    signatures = np.matmul(evecs ** 2, gaussian_kernel)  # [n, k], [k, e] -> [n, e]
    scale = np.sum(gaussian_kernel, axis=0, keepdims=True)  # [1, e]
    signatures /= scale

    return signatures


def heat_kernel_signature(evecs, evals, T):
    # Computes the heat kernel signature according to the spectrum of a graph operator(e.g., Laplacian).
    # The signature was introduced in the paper of J.Sun, M.Ovsjanikov and L.Guibas(2009)
    # "A Concise and Provably Informative Multi-Scale Signature-Based on Heat Diffusion."
    #
    # Input: evecs - [n, k] Eigenvectors of a graph operator arranged as columns.n denotes the number of nodes of the graph.
    # evals - [k] corresponding eigenvalues
    # T - [t] time values over which the kernel is evaluated
    #
    # Output:
    # signatures - [n, t] matrix with the values of the HKS for different T in its columns

    low_pass_filter = np.exp(-np.matmul(evals.reshape(-1, 1), T.reshape(1, -1)))  # [k, t]
    signatures = np.matmul(evecs ** 2, low_pass_filter)   #  [n, t]
    scale = np.sum(low_pass_filter, axis=0, keepdims=True)  # [1, t]
    signatures /= scale

    return signatures


def compute_mesh_info(v, f, hks_samples=100, wks_samples=100, num_eigs=100):

    L = -igl.cotmatrix(v, f)
    M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    evals, evecs = scipy.sparse.linalg.eigs(L, k=num_eigs, M=M, sigma=-1e-5)

    evals, evecs = np.real(evals[1:]), np.real(evecs[:, 1:])

    heat_time = energy_sample_generator('log_sampled', evals[0], evals[-1], hks_samples)
    hks_sig = heat_kernel_signature(evecs, evals, heat_time)

    energies, sigma = energy_sample_generator('log_linear', evals[0], evals[-1], wks_samples)
    wks_sig = wave_kernel_signature(evecs, evals, energies, sigma)

    return np.concatenate([hks_sig, wks_sig], axis=-1)


def load_mesh_info(path, compute_feature=True):
    v, f = igl.read_triangle_mesh(path)

    mesh_info = {'name': path.split('/')[-1].split('.')[-2],
                 'vertices': v,
                 'faces': f}

    if compute_feature:
        node_feature = compute_mesh_info(v, f)
        mesh_info['feature'] = node_feature

    return mesh_info

def compute_all_pair_normalized_geodesics(mesh, vids):

    geo = geodesic.PyGeodesicAlgorithmExact(mesh['vertices'], mesh['faces'])

    # Compute the longest distance on the mesh.
    # Find the farthest point 'p' from any point.
    dists, _ = geo.geodesicDistances([0], None)
    p_idx = np.argmax(dists)

    # Find the farthest point from point 'p'.
    dists, _ = geo.geodesicDistances([p_idx], None)
    max_dist = np.max(dists)

    num_vids = len(vids)
    all_dists = np.zeros((num_vids, num_vids))
    for i, vid in enumerate(vids):
        dists, _ = geo.geodesicDistances([vid], vids)
        all_dists[i] = dists / max_dist

    # Symmetrize.
    pair_geods = 0.5 * (all_dists + all_dists.T)

    # Ensure that the maximum normalized distance is zero.
    pair_geods[pair_geods > 1] = 1

    return pair_geods


def parse_args():
    from argparse import ArgumentParser
    parser = ArgumentParser()
    parser.add_argument('--path', type=str)
    return parser.parse_args()


def main(args):
    mesh_info = load_mesh_info(args.path)


if __name__ == '__main__':
    args = parse_args()
    main(args)


**1. Question a)**

In [11]:
def visualize_mat(mat, title, output_path):
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    cax = ax.matshow(mat)
    fig.colorbar(cax)
    ax.set_title(title)
    ax.set_aspect('equal')
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path)
    plt.close(fig)

import os
import numpy as np
import scipy.optimize
from google.colab import drive
drive.mount('/content/drive')


# Paths (adjust if needed)
mesh_dir = '/content/drive/MyDrive/cs 233/hmwk03_code_data/data_p2/meshes/'
landmark_filepath = '/content/drive/MyDrive/cs 233/hmwk03_code_data/data_p2/landmark_vids.txt'
output_dir = '/content/drive/MyDrive/cs 233/hmwk03_code_data/outputs_p2/desc_matching'

os.makedirs(output_dir, exist_ok=True)

# Load landmark vertex IDs and names
landmark_vids, landmark_names = load_landmark_file(landmark_filepath)
num_landmarks = len(landmark_vids)

# Load template mesh (with descriptors)
template_mesh_filepath = os.path.join(mesh_dir, 'template', 'mesh000.obj')
template_mesh_info = load_mesh_info(template_mesh_filepath, compute_feature=True)

# Load all test meshes (with descriptors)
test_mesh_dir = os.path.join(mesh_dir, 'test')
test_mesh_filepaths = [os.path.join(test_mesh_dir, fname)
                       for fname in os.listdir(test_mesh_dir)
                       if fname.endswith('.obj')]
test_mesh_infos = [load_mesh_info(fpath, compute_feature=True) for fpath in test_mesh_filepaths]
num_test_meshes = len(test_mesh_infos)

# Initialize cumulative correspondence matrix
C_acc = np.zeros((num_landmarks, num_landmarks))

for k in range(num_test_meshes):
    test_mesh_info = test_mesh_infos[k]
    test_name = test_mesh_info['name']
    print(test_name)

    # Extract descriptors of landmarks only
    template_feats = template_mesh_info['feature'][landmark_vids]  # (20, D)
    test_feats = test_mesh_info['feature'][landmark_vids]          # (20, D)

    # Compute pairwise L2 distances (cost matrix)
    cost_matrix = np.linalg.norm(template_feats[:, None, :] - test_feats[None, :, :], axis=-1)

    # Solve assignment problem (Hungarian algorithm)
    row_ind, col_ind = scipy.optimize.linear_sum_assignment(cost_matrix)

    # Build binary correspondence matrix
    C = np.zeros((num_landmarks, num_landmarks))
    for r, c in zip(row_ind, col_ind):
        C[r, c] = 1

    # Accumulate
    C_acc += C

    # Save binary matrix visualization
    visualize_mat(C, test_name, os.path.join(output_dir, f'{test_name}.png'))

# Compute and report accuracy
accuracy = np.sum(np.diagonal(C_acc)) / np.sum(C_acc)
print(f'Overall accuracy = {accuracy:.4f}')

# Save overall frequency matrix
visualize_mat(C_acc, 'All frequency', os.path.join(output_dir, 'all_frequency.png'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mesh006
mesh056 (1)
mesh019 (1)
mesh015
mesh019
mesh061 (1)
mesh056
mesh064 (1)
mesh026
mesh065 (1)
mesh011
mesh041 (1)
mesh052
mesh042
mesh058 (1)
mesh061
mesh065
mesh010
mesh058
mesh024
mesh036
mesh049 (1)
mesh005
mesh064
mesh049
mesh002
mesh004
mesh041
Overall accuracy = 0.7464


Overall accuracy is 0.7464, see all the matrix visualization pictures in the attached output_p2/desc_matching file.

See any failure patterns?

Yes, such as

1. Left vs Right limbs
2. Symmetric body parts (e.g., left/right hands, feet, elbows)


Why?

Because HKS and WKS are isometry-invariant but not orientation-aware:
1. Two symmetric points on opposite limbs can have nearly identical descriptors.
2. The signature depends on intrinsic structure, not global pose or handedness.



**2. Question b)**

In [6]:
###mrf:
## Python reimplementation of the algorithm described in the paper:

#  Marius Leordeanu and Martial Hebert,
#  Efficient MAP approximation for dense energy functions,
#  International Conference on Machine Learning, May, 2006.


# Utility:

# used for Maximum A Posteriori (MAP) labeling (discrete inference) problems
# where the task is to assign to each node i a label a
# such assignment is represented by a vector x, indexed by ia,
# such that x_{ia} = 1 if node i is labeled with a and 0 otherwise
# the goal is to maximize the labeling score x'Mx + Dx

# Input:

#       M: [N, N] matrix with pairwise potentials
#       D: [N] vector with unary potentials
#       node_indices: [N], node_indices[ia] = i, indicates the node x[ia] corresponds to
#       label_indices: [N], label_indices[ia] = a, indicates the label x[ia] corresponds to
#       iterEigen: nr of iterations of the initial stage (approx 30)
#       iterClimb: nr of iterations of the final stage   (approx 200)

import numpy as np


eps = 1e-6
def mrf(M, D, node_indices, label_indices, iterEigen, iterClimb):

    n = len(M)
    num_nodes = np.max(node_indices) + 1

    x = np.ones(n)

    # the indices in x that correspond to node j
    x_idx = [np.where(node_indices == j)[0] for j in range(num_nodes)]

    ## Stage 1: obtain the starting point using the normalized power / eigen method
    # this finds the global maximum to the relaxed problem

    for _ in range(iterEigen):
        # x = np.matmul(2 * M, x) + D
        x = np.matmul(M, x)
        for j in range(num_nodes):
            x[x_idx[j]] /= np.sqrt((x[x_idx[j]] ** 2).sum() + eps)


    # now start from x, project in on the simplex, and keep climbing using a similar iterative method

    for j in range(num_nodes):
        x[x_idx[j]] /= np.sum(x[x_idx[j]]) + eps

    ## Stage 2: climb until convergence

    step = 1 / iterClimb
    beta = np.arange(1, 0.01, -step)

    iterClimb = len(beta)
    beta = 1 / beta

    for i in range(iterClimb):
        prev_x = x
        for j in range(num_nodes):
            x[x_idx[j]] = prev_x[x_idx[j]] * ((np.matmul(2 * M[x_idx[j]], prev_x) + D[x_idx[j]]) ** beta[i])
            x[x_idx[j]] /= np.sum(x[x_idx[j]]) + eps


    sol = np.zeros(n)
    labels = []
    for j in range(num_nodes):
        index = np.argmax(x[x_idx[j]])
        index = x_idx[j][index]
        sol[index] = 1
        labels.append(label_indices[index])

    score = np.matmul(sol, np.matmul(M, sol)) + np.dot(D, sol)

    return sol, score, labels

In [10]:

for k in range(num_test_meshes):
    test_mesh_info = test_mesh_infos[k]
    test_name = test_mesh_info['name']
    print(test_name)

    test_geod_dists = compute_all_pair_normalized_geodesics(test_mesh_info, landmark_vids)
    template_geod_dists = compute_all_pair_normalized_geodesics(template_mesh_info, landmark_vids)
    output_dir = '/content/drive/MyDrive/cs 233/hmwk03_code_data/outputs_p2/pairwise_matching'
    # MRF Setup
    sigma = 0.05
    V = num_landmarks
    N = V * V  # each node can take any label (20 x 20)

    M = np.zeros((N, N))
    D = np.random.rand(N) * 0.01  # very small random unary to break symmetry
    node_indices = np.zeros(N, dtype=int)
    label_indices = np.zeros(N, dtype=int)

    for i in range(V):
        for j in range(V):
            idx = i * V + j
            node_indices[idx] = i
            label_indices[idx] = j

    for i in range(V):
        for j in range(V):
            for k_ in range(V):
                for l_ in range(V):
                    idx1 = i * V + j
                    idx2 = k_ * V + l_
                    diff = template_geod_dists[i, k_] - test_geod_dists[j, l_]
                    weight = np.exp(-(diff ** 2) / (2 * sigma))
                    M[idx1, idx2] = weight

    # Solve with MRF
    solution, score, labels = mrf(M, D, node_indices, label_indices,
                                   iterEigen=30, iterClimb=200)

    # Build binary correspondence matrix
    C = np.zeros((num_landmarks, num_landmarks))
    for i in range(V):
        C[i, labels[i]] = 1

    C_acc += C
    visualize_mat(C, test_name, os.path.join(output_dir, f'{test_name}.png'))


mesh006
mesh056 (1)
mesh019 (1)
mesh015
mesh019
mesh061 (1)
mesh056
mesh064 (1)
mesh026
mesh065 (1)
mesh011
mesh041 (1)
mesh052
mesh042
mesh058 (1)
mesh061
mesh065
mesh010
mesh058
mesh024
mesh036
mesh049 (1)
mesh005
mesh064
mesh049
mesh002
mesh004
mesh041


In [8]:

# Compute and report accuracy
accuracy = np.sum(np.diagonal(C_acc)) / np.sum(C_acc)
print(f'Overall accuracy = {accuracy:.4f}')


Overall accuracy = 0.7134


See the matrix plots in the attached output_p2/pairwise_matching files.

Discussion for part b):

In part (b), the MRF-based geodesic matching achieved an accuracy of 71.34%, slightly lower than the 74.64% from descriptor matching in part (a). While geodesic distances offer a more global and symmetry-aware perspective, their effectiveness is limited by numerical noise, pose similarity, and the small number of landmarks. Additionally, MRF optimization can struggle when different poses yield similar geodesic patterns. Despite its theoretical strengths, geodesic matching may require more landmarks or careful tuning to outperform simpler descriptor-based methods.